In [1]:
import pymongo

In [2]:
con_str = "localhost:27017"
db = pymongo.MongoClient(con_str, serverSelectionTimeoutMS = 10000)
try:
    print(db.server_info())
except Exception:
    print("connect failed!")

{'version': '6.0.0', 'gitVersion': 'e61bf27c2f6a83fed36e5a13c008a32d563babe2', 'targetMinOS': 'Windows 7/Windows Server 2008 R2', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [6, 0, 0, 0], 'openssl': {'running': 'Windows SChannel'}, 'buildEnvironment': {'distmod': 'windows', 'distarch': 'x86_64', 'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.31.31107 for x64', 'ccflags': '/nologo /WX /FImongo/platform/basic.h /EHsc /W3 /wd4068 /wd4244 /wd4267 /wd4290 /wd4351 /wd4355 /wd4373 /wd4800 /wd4251 /wd4291 /we4013 /we4099 /we4930 /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /permissive- /Zc:__cplusplus /Zc:sizedDealloc /volatile:iso /diagnostics:caret /std:c++17 /Gw /Gy /Zc:inline', 'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.31.31107 for x64', 'cxxflags': '/TP', 'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF', 'target_arch': 'x86_64', 'target_os': 'windows', 'cppdefine

In [4]:
# 查询数据库中所有数据库的名称
db_names = db.list_database_names()
db_names

['admin',
 'config',
 'db_Weibo_SH0331',
 'db_Weibo_SH_0331_All',
 'db_tvcgR3',
 'db_weiboDB',
 'local',
 'test',
 'weiboInfoAll']

In [4]:
# 导入所有数据
def load_data_fromCsv(path):

    weiboDataByOpen8Csv = list()  # ③ 用open和csv.reader读取得到的webo数据

    with open(path, encoding='utf-8-sig') as f:

        next(f)  # 跳过标题行
        for row in csv.reader(f, skipinitialspace=True):

            item = row[0]
            index = row[1]
            title8content = row[2]
            OrgOrFwd = row[3]
            date = row[4]
            orgContent = row[5]
            AuthType = row[6]
            SourceRegion = row[7]
            city = row[8]
            gender = row[9]
            MD5_author = row[10]
            MD5_author_ID = row[11]
            MD5_mid = row[12]
            author = row[13]
            mediumType = row[14]

            doc = dict()  # 创建表示文档的字典（一个文档就算1条数据）

            doc['item'] = item
            doc['index'] = index
            doc['title8content'] = title8content
            doc['OrgOrFwd'] = OrgOrFwd
            doc['date'] = date
            doc['orgContent'] = orgContent
            doc['AuthType'] = AuthType
            doc['SourceRegion'] = SourceRegion
            doc['city'] = city
            doc['gender'] = gender
            doc['MD5_author'] = MD5_author
            doc['MD5_author_ID'] = MD5_author_ID
            doc['MD5_mid'] = MD5_mid
            doc['author'] = author
            doc['mediumType'] = mediumType

            weiboDataByOpen8Csv.append(doc)
    return weiboDataByOpen8Csv

In [5]:
import csv
import pymongo

if __name__ == '__main__':

    weboData = load_data_fromCsv(r'E:\sns_data\Weibo_SH_0331_All\Weibo_SH_0331.csv')

    # print(weboData[0])
    
    # 连接数据库
    db = pymongo.MongoClient("mongodb://localhost:27017/")
    db_weibo_SH = db["db_Weibo_SH0331"]

    # 数据库里的表（集合）
    col_weiboSH0331 = db_weibo_SH["col_Weibo_SH0331"]
    
    col_weiboSH0331.insert_many(weboData)

In [14]:
# 打印两条记录看看导入状况和数据状态
egRec1 = col_weiboSH0331.find_one()
print(egRec1['_id'])
print(egRec1['date'])
# print(egRec1['title8content'])
# print(egRec1['orgContent'])


egRec2 = col_weiboSH0331.find_one({'item' : "904075"})
print(egRec2['_id'])
print(egRec2['date'])
print(type(egRec2))

62fc6990f8180a03b21c2a79
2022-03-31 00:00:00
62fc69b0f8180a03b229f604
2022-03-31 19:51:00
<class 'dict'>


In [15]:
# 查看当前集合现存的所有字段
print(col_weiboSH0331.find_one().keys())

dict_keys(['_id', 'item', 'index', 'title8content', 'OrgOrFwd', 'date', 'orgContent', 'AuthType', 'SourceRegion', 'city', 'gender', 'MD5_author', 'MD5_author_ID', 'MD5_mid', 'author', 'mediumType'])


### 下面分时段处理，首先要新建一个时段（列名），保存每天记录所在的时段（单位：1 Hour）
> 后来发现，不用新建字段，只需要在处理数据的时候，直接操作字段即可（系统会新建不存在的字段）

In [16]:
# 新建一个字段（timePeriod）
isExistsTimePeriod = col_weiboSH0331.find_one({"timePeriod": {'$exists': True}})

if not isExistsTimePeriod:
    for c in col_weiboSH0331.find():
        col_weiboSH0331.update_one({'_id': c['_id']}, {'$set': {'timePeriod': list()}})
    print("不存在，已新增！")
else:
    print("已存在，未改动！")

不存在，已新增！


In [17]:
# （新建timePeriod之后，再次）查看当前集合现存的所有字段
print(col_weiboSH0331.find_one().keys())

# 打印当前集合的记录总数
print(col_weiboSH0331.count_documents({}))  

dict_keys(['_id', 'item', 'index', 'title8content', 'OrgOrFwd', 'date', 'orgContent', 'AuthType', 'SourceRegion', 'city', 'gender', 'MD5_author', 'MD5_author_ID', 'MD5_mid', 'author', 'mediumType', 'timePeriod'])
1055876


### 收获总结
> ① 想要把数据处理后，保存过程中产生的变量，不需要新建字段，直接操作对应集合的“新字段”就可以，系统会默认创建 <br>
> ② Markdown的语法记录之一：内置换行可用HTML标签`<br>` <br>
> ③ pymongo 和 MongoDB的语法不一样，mongodb的操作手册，并不能精准解决pymongo的问题。查看帮助手册的通用方法如下：
> * **首先**，查看工具包的版本：`pip show pymongo` -> `Version: 4.2.0`
> * **然后**，根据工具包版本搜索关键词（关键词要指向明确，比如pymongo，不是mongo，不是mongoDB）: `pymongo 4.2 文档`
> * **最后**，查手册，还是需要精准搜索，比如**我想查询mongodb数据库内部的集合/表的记录书目 <br>
（在网上检索一番后，或者自己根据经验，输入英文关键词：count）**
> * 得到手册中的指向解决方案：`.count_documents({})`，即最新的查看集合中记录总数的语句 <br>